In [ ]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()

In [ ]:
# This project will be one that uses a quantum circuit to generate a random password
from qiskit import QuantumCircuit, transpile, assemble, Aer, IBMQ, execute
from qiskit.circuit import QuantumRegister, ClassicalRegister
from qiskit.providers.aer import AerSimulator

# Function that converts a list of bits
# into decimal. MSB first
def bits_to_dec(bits):
    dec = 0
    iter_count = 0
    for x in range(len(bits),0,-1):
        dec += int(bits[x-1]) * (2**(x-1))
        iter_count += 1
    return dec


# Function that selects a random character from the available character groups
# and from the provided index
def select_char(group_num, index_num):
    # Choose the group depending on the group number
    # The index needs a modulo operator since not all groups have the same size
    char_to_return = ''
    if group_num == 0:
        index = index_num % len(uppercase_alpha)
        char_to_return = uppercase_alpha[index]
    elif group_num == 1:
        index = index_num % len(lowercase_alpha) 
        char_to_return = lowercase_alpha[index]
    elif group_num == 2:
        index = index_num % len(numbers)
        numbers[index]
    else:
        index = index_num % len(special_symbols)
        char_to_return = special_symbols[index]
    return char_to_return
        
    
# Define lists of characters for passwords
uppercase_alpha = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
lowercase_alpha = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
numbers = ['1', '2', '3', '4', '5', '6', '7', '8', '0']
special_symbols = ['!', ':', ';', '?', '/', '.', ',', '#', '@', '&', '*', '-', '^', '$', '+', '=']

# Define variables that will be used to output the password
password = ''
group_bits = 0
index_bits = 0

# The quantum circuit will have 7 lines. 
# The first 2 lines will determine which group to use, the next 5 lines will determine which element of the list to use

qr = QuantumRegister(7)
cr = ClassicalRegister(7)
qc = QuantumCircuit(qr, cr)

qgroup0, qgroup1, qindex0, qindex1, qindex2, qindex3, qindex4 = qr
bgroup0, bgroup1, bindex0, bindex1, bindex2, bindex3, bindex4 = cr

# Apply a Hadamard gate to all qubits to introduce randomness
qc.h(qr)

# measure all the qubits
qc.measure([qgroup0, qgroup1, qindex0, qindex1, qindex2, qindex3, qindex4], [bgroup0, bgroup1, bindex0, bindex1, bindex2, bindex3, bindex4])

good_input = False
while good_input is False:
    try:
        pass_length = input("Enter password length (at least 8 characters long): ")
        if pass_length <= 8:
            break;
    except:
        print("Try inputting another number")

for i in range(int(pass_length)):
    # Run the job to get the 7 bits
    backend = Aer.get_backend('qasm_simulator')
    counts = execute(qc, backend, shots = 1).result().get_counts()

    # Check the value returned by getting the key
    seven_bits = []
    for key in counts.keys():
        seven_bits.append(key)

    bit_num = seven_bits[0]
    
    # Convert the bits into decimal, and acquire the letter to be appended into the password
    group_num = bits_to_dec([bit_num[1], bit_num[0]])
    index_num = bits_to_dec([bit_num[6], bit_num[5], bit_num[4], bit_num[3], bit_num[2]])
    password += select_char(group_num, index_num)
    
print("Your password is: " + password)
# Access the bits and calculate the stuff
        
qc.draw(output="mpl")
# Construct a 4-bit quantum circuit